In [90]:
import sys
import heapq
from collections import defaultdict

In [91]:
sys.path.append("../")

In [92]:
file_path = 'input_ag.txt'

In [93]:
maze_map = []
with open(file_path, 'r') as file:
    for line in file:
        maze_map.append(line.strip('\n'))

In [94]:
maze_map = [list(row) for row in maze_map]

In [95]:
def pretty_print_map(map_grid):
    for row in map_grid:
        print(''.join(row))

In [96]:
pretty_print_map(maze_map)

#############################################################################################################################################
#...#.................................#...#...#...............#...............#.........#...............#...#.....#.........#.....#........E#
#.###.#.#####.###########.#.#.#####.#.#.#.#.#.#.###.#####.#.###.#########.#.#.#.#.#####.###.#######.#####.#.#.#.#.#######.#.#.#.###.###.###.#
#.........#...#.........#.#.#.........#.#...#.....#.....#.#.....#.........#.#...#.#...#...#.....#...#.........................#.....#...#.#.#
#######.###.#####.###.###.#########.#.#.###############.###.#####.#########.#.###.###.###.#####.#.#.#.#########.###.#.#.#.###.#######.###.#.#
#.........#.....#...#.#...#.........#...#...#...#.............#...#.#.....#.#...#...#...#.......#...#.....#...#...#...#.#...#.....#...#...#.#
#.#######.#.###.#.#.###.###.#######.#.#.#.#.#.###.###.#.#####.#.###.#.#.#.#.#.#.###.#.#.#.#######.#.#####.#.#.###.#####.###.###.###.###.#.#.#
#.#...

In [97]:
example_maze = [
    '###############',
    '#.......#....E#',
    '#.#.###.#.###.#',
    '#.....#.#...#.#',
    '#.###.#####.#.#',
    '#.#.#.......#.#',
    '#.#.#####.###.#',
    '#...........#.#',
    '###.#.#####.#.#',
    '#...#.....#.#.#',
    '#.#.#.###.#.#.#',
    '#.....#...#.#.#',
    '#.###.#.#.#.#.#',
    '#S..#.....#...#',
    '###############'
]

In [98]:
example_maze2 = [
'#################',
'#...#...#...#..E#',
'#.#.#.#.#.#.#.#.#',
'#.#.#.#...#...#.#',
'#.#.#.#.###.#.#.#',
'#...#.#.#.....#.#',
'#.#.#.#.#.#####.#',
'#.#...#.#.#.....#',
'#.#.#####.#.###.#',
'#.#.#.......#...#',
'#.#.###.#####.###',
'#.#.#...#.....#.#',
'#.#.#.#####.###.#',
'#.#.#.........#.#',
'#.#.#.#########.#',
'#S#.............#',
'#################',
]

In [99]:
example_maze = [list(row) for row in example_maze]

In [100]:
def rotate_clockwise(current_direction):
    if current_direction == (0,1):
        return (1,0)
    elif current_direction == (1,0):
        return (0,-1)
    elif current_direction == (0,-1):
        return (-1,0)
    elif current_direction == (-1,0):
        return (0,1)

In [101]:
def rotate_anticlockwise(current_direction):
    if current_direction == (0,1):
        return (-1,0)
    elif current_direction == (1,0):
        return (0,1)
    elif current_direction == (0,-1):
        return (1,0)
    elif current_direction == (-1,0):
        return (0,-1)

In [102]:
def solve_maze(maze):
    # Directions in (row_delta, col_delta) order
    directions = [(0,1), (1,0), (0,-1), (-1,0)]
    
    # Find S and E
    start = None
    exit = None
    for i in range(len(maze)):
        for j in range(len(maze[0])):
            if maze[i][j] == 'S':
                start = (i, j)
            elif maze[i][j] == 'E':
                exit = (i, j)
    
    # To reconstruct the path, we keep track of parents
    parents = {}
    
    # Initiaize the costs at infinity
    dist = {}
    for row in range(len(maze)):
        for col in range(len(maze[0])):
            for direction in directions:
                dist[(row, col, direction)] = float('inf')

    # Priority queue of (cost, row, col, (dr, dc))
    pq = []
    
    # We assume we start facing East (0,1). If you allow any initial direction,
    # you'd push all 4 directions with cost=0, or cost=1000 if turning is required.
    initial_direction = (0, 1)
    parents[(start[0], start[1], initial_direction)] = None  # No parent for the start
    dist[(start[0], start[1], initial_direction)] = 0
    heapq.heappush(pq, (0, start[0], start[1], initial_direction))# Initialize the priority queue
    
    while pq:
        current_cost, row, col, direction = heapq.heappop(pq)
        
        # If this cost is already worse than the best found, skip
        if current_cost > dist[(row, col, direction)]:
            continue
        
        # If we've reached the end, return the cost
        if (row, col) == exit:
            print(f"Found path with score {current_cost}")
            path = reconstruct_path(parents, (row, col, direction))
            return current_cost, path
        
        # 1) Move Forward
        nr = row + direction[0]
        nc = col + direction[1]
        if 0 <= nr < len(maze) and 0 <= nc < len(maze[0]):
            if maze[nr][nc] != '#':  # not a wall
                new_cost = current_cost + 1
                if new_cost < dist[(nr, nc, direction)]:
                    dist[(nr, nc, direction)] = new_cost
                    parents[(nr, nc, direction)] = (row, col, direction)
                    heapq.heappush(pq, (new_cost, nr, nc, direction))
        
        # 2) Turn Clockwise
        d_cw = rotate_clockwise(direction)
        cost_cw = current_cost + 1000
        if cost_cw < dist[(row, col, d_cw)]:
            dist[(row, col, d_cw)] = cost_cw
            parents[(row, col, d_cw)] = (row, col, direction)
            heapq.heappush(pq, (cost_cw, row, col, d_cw))
        
        # 3) Turn Counterclockwise
        d_ccw = rotate_anticlockwise(direction)
        cost_ccw = current_cost + 1000
        if cost_ccw < dist[(row, col, d_ccw)]:
            dist[(row, col, d_ccw)] = cost_ccw
            parents[(row, col, d_ccw)] = (row, col, direction)
            heapq.heappush(pq, (cost_ccw, row, col, d_ccw))
    
    # If we exit the loop without returning, there's no path
    return None, None

In [103]:
def reconstruct_path(parents, end_state):
    """
    Reconstructs the path (row, col, direction) from the end_state
    back to the start by following the 'parents' dictionary.
    """
    path = []
    current_state = end_state
    while current_state is not None:
        path.append(current_state)
        current_state = parents[current_state]
    path.reverse()
    return path

In [104]:
solve_maze(example_maze)

Found path with score 7036


(7036,
 [(13, 1, (0, 1)),
  (13, 1, (-1, 0)),
  (12, 1, (-1, 0)),
  (11, 1, (-1, 0)),
  (11, 1, (0, 1)),
  (11, 2, (0, 1)),
  (11, 3, (0, 1)),
  (11, 4, (0, 1)),
  (11, 5, (0, 1)),
  (11, 5, (-1, 0)),
  (10, 5, (-1, 0)),
  (9, 5, (-1, 0)),
  (8, 5, (-1, 0)),
  (7, 5, (-1, 0)),
  (7, 5, (0, 1)),
  (7, 6, (0, 1)),
  (7, 7, (0, 1)),
  (7, 8, (0, 1)),
  (7, 9, (0, 1)),
  (7, 10, (0, 1)),
  (7, 11, (0, 1)),
  (7, 11, (1, 0)),
  (8, 11, (1, 0)),
  (9, 11, (1, 0)),
  (10, 11, (1, 0)),
  (11, 11, (1, 0)),
  (12, 11, (1, 0)),
  (13, 11, (1, 0)),
  (13, 11, (0, 1)),
  (13, 12, (0, 1)),
  (13, 13, (0, 1)),
  (13, 13, (-1, 0)),
  (12, 13, (-1, 0)),
  (11, 13, (-1, 0)),
  (10, 13, (-1, 0)),
  (9, 13, (-1, 0)),
  (8, 13, (-1, 0)),
  (7, 13, (-1, 0)),
  (6, 13, (-1, 0)),
  (5, 13, (-1, 0)),
  (4, 13, (-1, 0)),
  (3, 13, (-1, 0)),
  (2, 13, (-1, 0)),
  (1, 13, (-1, 0))])

In [105]:
solve_maze(example_maze2)

Found path with score 11048


(11048,
 [(15, 1, (0, 1)),
  (15, 1, (-1, 0)),
  (14, 1, (-1, 0)),
  (13, 1, (-1, 0)),
  (12, 1, (-1, 0)),
  (11, 1, (-1, 0)),
  (10, 1, (-1, 0)),
  (9, 1, (-1, 0)),
  (8, 1, (-1, 0)),
  (7, 1, (-1, 0)),
  (6, 1, (-1, 0)),
  (5, 1, (-1, 0)),
  (5, 1, (0, 1)),
  (5, 2, (0, 1)),
  (5, 3, (0, 1)),
  (5, 3, (1, 0)),
  (6, 3, (1, 0)),
  (7, 3, (1, 0)),
  (8, 3, (1, 0)),
  (9, 3, (1, 0)),
  (10, 3, (1, 0)),
  (11, 3, (1, 0)),
  (12, 3, (1, 0)),
  (13, 3, (1, 0)),
  (14, 3, (1, 0)),
  (15, 3, (1, 0)),
  (15, 3, (0, 1)),
  (15, 4, (0, 1)),
  (15, 5, (0, 1)),
  (15, 5, (-1, 0)),
  (14, 5, (-1, 0)),
  (13, 5, (-1, 0)),
  (12, 5, (-1, 0)),
  (11, 5, (-1, 0)),
  (11, 5, (0, 1)),
  (11, 6, (0, 1)),
  (11, 7, (0, 1)),
  (11, 7, (-1, 0)),
  (10, 7, (-1, 0)),
  (9, 7, (-1, 0)),
  (9, 7, (0, 1)),
  (9, 8, (0, 1)),
  (9, 9, (0, 1)),
  (9, 10, (0, 1)),
  (9, 11, (0, 1)),
  (9, 11, (-1, 0)),
  (8, 11, (-1, 0)),
  (7, 11, (-1, 0)),
  (7, 11, (0, 1)),
  (7, 12, (0, 1)),
  (7, 13, (0, 1)),
  (7, 14, (0, 1)),

In [106]:
solve_maze(maze_map)

Found path with score 85432


(85432,
 [(139, 1, (0, 1)),
  (139, 1, (-1, 0)),
  (138, 1, (-1, 0)),
  (137, 1, (-1, 0)),
  (137, 1, (0, 1)),
  (137, 2, (0, 1)),
  (137, 3, (0, 1)),
  (137, 4, (0, 1)),
  (137, 5, (0, 1)),
  (137, 5, (-1, 0)),
  (136, 5, (-1, 0)),
  (135, 5, (-1, 0)),
  (134, 5, (-1, 0)),
  (133, 5, (-1, 0)),
  (133, 5, (0, 1)),
  (133, 6, (0, 1)),
  (133, 7, (0, 1)),
  (133, 8, (0, 1)),
  (133, 9, (0, 1)),
  (133, 10, (0, 1)),
  (133, 11, (0, 1)),
  (133, 12, (0, 1)),
  (133, 13, (0, 1)),
  (133, 14, (0, 1)),
  (133, 15, (0, 1)),
  (133, 15, (1, 0)),
  (134, 15, (1, 0)),
  (135, 15, (1, 0)),
  (135, 15, (0, 1)),
  (135, 16, (0, 1)),
  (135, 17, (0, 1)),
  (135, 18, (0, 1)),
  (135, 19, (0, 1)),
  (135, 19, (-1, 0)),
  (134, 19, (-1, 0)),
  (133, 19, (-1, 0)),
  (133, 19, (0, -1)),
  (133, 18, (0, -1)),
  (133, 17, (0, -1)),
  (133, 17, (-1, 0)),
  (132, 17, (-1, 0)),
  (131, 17, (-1, 0)),
  (131, 17, (0, -1)),
  (131, 16, (0, -1)),
  (131, 15, (0, -1)),
  (131, 15, (-1, 0)),
  (130, 15, (-1, 0)),
  

## Second star

In [107]:
def solve_maze_all_paths(maze):
    # Directions in (row_delta, col_delta) order
    directions = [(0,1), (1,0), (0,-1), (-1,0)]
    
    # Find S and E in the maze
    start = None
    exit_ = None
    for i in range(len(maze)):
        for j in range(len(maze[0])):
            if maze[i][j] == 'S':
                start = (i, j)
            elif maze[i][j] == 'E':
                exit_ = (i, j)
    
    # dist[(row, col, direction)] = cost to reach this state
    dist = {}
    for row in range(len(maze)):
        for col in range(len(maze[0])):
            for d in directions:
                dist[(row, col, d)] = float('inf')
    
    # parents[(r, c, d)] = list of predecessor states (r0, c0, d0)
    # that lead to (r, c, d) at the *same* minimal cost
    parents = defaultdict(list)
    
    # Priority queue of (cost, row, col, (dr, dc))
    pq = []
    
    # We assume we start facing East (0,1).
    initial_direction = (0, 1)
    dist[(start[0], start[1], initial_direction)] = 0
    # Put a sentinel to indicate that the start has no predecessor
    # but we store an empty list for it in parents
    parents[(start[0], start[1], initial_direction)] = []

    heapq.heappush(pq, (0, start[0], start[1], initial_direction))
    
    # 1) Dijkstra to find the minimal cost for each state
    while pq:
        current_cost, row, col, direction = heapq.heappop(pq)
        
        # If this cost is worse than the best known, skip
        if current_cost > dist[(row, col, direction)]:
            continue
        
        # Move 1: forward
        nr, nc = row + direction[0], col + direction[1]
        if 0 <= nr < len(maze) and 0 <= nc < len(maze[0]):
            if maze[nr][nc] != '#':  # not a wall
                new_cost = current_cost + 1
                if new_cost < dist[(nr, nc, direction)]:
                    dist[(nr, nc, direction)] = new_cost
                    parents[(nr, nc, direction)] = [(row, col, direction)]
                    heapq.heappush(pq, (new_cost, nr, nc, direction))
                elif new_cost == dist[(nr, nc, direction)]:
                    # Another equally optimal way to get (nr, nc, direction)
                    parents[(nr, nc, direction)].append((row, col, direction))
        
        # Move 2: turn clockwise
        d_cw = rotate_clockwise(direction)
        cost_cw = current_cost + 1000
        if cost_cw < dist[(row, col, d_cw)]:
            dist[(row, col, d_cw)] = cost_cw
            parents[(row, col, d_cw)] = [(row, col, direction)]
            heapq.heappush(pq, (cost_cw, row, col, d_cw))
        elif cost_cw == dist[(row, col, d_cw)]:
            # Another equally optimal way to get (row, col, d_cw)
            parents[(row, col, d_cw)].append((row, col, direction))
        
        # Move 3: turn counterclockwise
        d_ccw = rotate_anticlockwise(direction)
        cost_ccw = current_cost + 1000
        if cost_ccw < dist[(row, col, d_ccw)]:
            dist[(row, col, d_ccw)] = cost_ccw
            parents[(row, col, d_ccw)] = [(row, col, direction)]
            heapq.heappush(pq, (cost_ccw, row, col, d_ccw))
        elif cost_ccw == dist[(row, col, d_ccw)]:
            # Another equally optimal way to get (row, col, d_ccw)
            parents[(row, col, d_ccw)].append((row, col, direction))
    
    # 2) Find the minimal cost to reach the exit in *any* orientation
    # If unreachable, dist will remain inf for all directions
    best_cost = float('inf')
    best_orientations = []
    for d in directions:
        c = dist[(exit_[0], exit_[1], d)]
        if c < best_cost:
            best_cost = c
            best_orientations = [d]
        elif c == best_cost:
            best_orientations.append(d)
    
    if best_cost == float('inf'):
        # No path
        return None, []
    
    # 3) For each orientation that has the best cost, we backtrack to find all paths
    all_paths = []
    for d in best_orientations:
        # Reconstruct *all* paths from (exit_, d)
        partial_paths = backtrack_all_paths(parents, (exit_[0], exit_[1], d))
        all_paths.extend(partial_paths)
    
    # all_paths is a list of paths, where each path is [ (r,c,d), (r,c,d), ..., (r,c,d) ]
    return best_cost, all_paths


def backtrack_all_paths(parents, current_state):
    """
    Returns *all* possible paths from the 'start' (which has no parent)
    to 'current_state' by recursively exploring all parents of 'current_state'.
    
    parents is a dict of the form:
       parents[(r, c, d)] = [ (pr, pc, pd), ... ]
    where each predecessor leads to (r,c,d) with the same minimal cost path.
    """
    # If this state has no parents, we've reached the start
    # (In our code, the start was stored with parents[start] = [])
    if not parents[current_state]:
        # Return a list containing just a single path: [ current_state ]
        return [[current_state]]
    
    # Otherwise, gather all paths leading to each of the parents
    all_paths = []
    for parent_state in parents[current_state]:
        # Recursively get all paths from start to parent_state
        parent_paths = backtrack_all_paths(parents, parent_state)
        # For each parent path, append current_state
        for p_path in parent_paths:
            all_paths.append(p_path + [current_state])
    
    return all_paths

In [116]:
best_cost, all_paths = solve_maze_all_paths(maze_map)

In [117]:
best_cost

85432

In [118]:
unique_positions = set()

for path in all_paths:
    for (r, c, d) in path:
        unique_positions.add((r, c))

# Now we have all unique (row, col) cells across all paths
num_unique_locations = len(unique_positions)

print("Number of unique locations visited in all optimal paths:", num_unique_locations)

Number of unique locations visited in all optimal paths: 465
